# Portfolio Workflow Demo
This example demonstrates building a problem using `ProblemBuilder` and running an optimisation.

In [ ]:
import polars as pl
from ipywidgets import FloatSlider, Button, VBox, HBox
from ipydatagrid import DataGrid

from opt.instruments import InstrumentMap
from opt.risk import FactorRiskModel, FactorRiskModelData
from opt.costs import PowerLawCost
from opt.optimizer import optimize_portfolio
from opt.workflow import ProblemBuilder
from opt.portfolio import Portfolio

In [ ]:
positions = pl.read_csv('sample_positions.csv')
positions

In [ ]:
grid = DataGrid(positions)
grid

In [ ]:
n = len(positions)
imap = InstrumentMap.identity(n)
alpha = pl.Series('a', [0.1 + i*0.1 for i in range(n)]).to_numpy()
loadings = pl.eye(n).to_numpy()
frmd = FactorRiskModelData(loadings=loadings, factor_cov=pl.eye(n).to_numpy(), specific_var=0.1*pl.ones(n).to_numpy())
rm = FactorRiskModel(data=frmd)
portfolio = Portfolio(positions=positions['position'].to_numpy())

In [ ]:
builder = ProblemBuilder(risk_model=rm, instrument_map=imap, alpha_dec=alpha)
builder.with_start_from_portfolio(portfolio)
builder.add_bounds(range(n), 0.0, 0.3)
builder.add_gross_limit(2.0)
builder.add_net_bounds(0.9, 1.1)
builder.with_cost_model(PowerLawCost(scale=0.01))
problem = builder.build()

In [ ]:
res = optimize_portfolio(problem)
res